<a href="https://colab.research.google.com/github/sinjy1203/deep-learning/blob/master/handson_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import tensorflow as tf
import numpy as np
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [28]:
x_train, x_test = x_train.astype(np.float32).reshape(-1, 28*28), x_test.astype(np.float32).reshape(-1, 28*28)
y_train, y_test = y_train.astype(np.int32), y_test.astype(np.int32)
y_train.shape

(60000,)

In [81]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

tf.reset_default_graph()
tf.set_random_seed(42)
np.random.seed(42)
x = tf.placeholder(dtype=tf.float32, shape=(None, 784), name='x')
y = tf.placeholder(dtype=tf.int32, shape=(None), name='y')

layer1 = tf.layers.dense(x, 200, activation=tf.nn.relu, name='layer1')
layer2 = tf.layers.dense(layer1, 200, activation=tf.nn.relu, name='layer2')
layer3 = tf.layers.dense(layer2, 200, activation=tf.nn.relu, name='layer3')
layer4 = tf.layers.dense(layer3, 200, activation=tf.nn.relu, name='layer4')
layer5 = tf.layers.dense(layer4, 200, activation=tf.nn.relu, name='layer5')
logits = tf.layers.dense(layer5, 10)
pred = tf.nn.softmax(logits)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits=logits)
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.GradientDescentOptimizer(0.001)
train_op = optimizer.minimize(loss)

correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 101
batch_size = 50

with tf.Session() as sess:
  sess.run(init)
  for n_epoch in range(n_epochs):
    cost = 0.0
    for i in range(np.int(len(x_train) / batch_size)):
      xs = x_train[i * batch_size:i * batch_size + batch_size]
      ys = y_train[i * batch_size:i * batch_size + batch_size]
      _, cost_val = sess.run([train_op, loss], feed_dict={x: xs, y: ys})
      cost += cost_val / (len(x_train) / batch_size)
    if n_epoch % 10 == 0:
      print(n_epoch, cost)
  save_path = saver.save(sess, "/content/gdrive/My Drive/tmp/my_model3.ckpt")
# a = 0
# with tf.Session() as sess:
#   sess.run(init)
#   cost = 0
#   for epoch in range(n_epochs):
#         for x_batch, y_batch in shuffle_batch(x_train, y_train, batch_size):
#             _, cost_val = sess.run([train_op, loss], feed_dict={x: x_batch, y: y_batch})
#             #a += 1
#             cost += cost_val / 1200
#         if epoch % 10 == 0:
#           print(epoch,"  ", cost)
#   save_path = saver.save(sess, "/content/gdrive/My Drive/tmp/my_model3.ckpt")
  #print(a)
# with tf.Session() as sess:
#   sess.run(init)
#   for n_epoch in range(n_epochs):
#     _, cost = sess.run([train_op, loss], feed_dict={x: x_train, y: y_train})
#     if n_epoch % 10 == 0:
#       print(n_epoch, cost)
#   save_path = saver.save(sess, "/content/gdrive/My Drive/tmp/my_model3.ckpt")

0 0.8090774025201496
10 0.04074946763689995
20 0.010931224058191837
30 0.004242333644072765
40 0.002264588139081149
50 0.0014551819595612875
60 0.001048597557021747
70 0.0008080915473230261
80 0.0006516356432399571
90 0.0005424005499416275
100 0.000462710872168562


In [82]:
with tf.Session() as sess:
  saver.restore(sess, "/content/gdrive/My Drive/tmp/my_model3.ckpt")
  acc = sess.run(accuracy, feed_dict={x: x_test, y: y_test})
  print(acc)

INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/tmp/my_model3.ckpt
0.96
